# Imports

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import warnings

from tqdm import tqdm

In [2]:
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# Global Variables

In [3]:
URL = 'https://es.ign.com'
DATASET = 'test.csv'

# Data Processing

In [12]:
df = pd.read_csv(DATASET)
df.published_date = pd.to_datetime(df.published_date)
(df.published_date.dt.month == 9).sum()

60

In [4]:
page = requests.get(URL, verify = False)

/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es.ign.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
soup = BeautifulSoup(page.text, 'html.parser')
res = {'title':[], 'link':[]}

articles = soup.find_all('article')
for elt in tqdm(articles) : 
    res['title'].append(elt.find('h3').get_text())
    res['link'].append(str(elt.find('a', href=True)).split('"')[3])

100%|██████████| 80/80 [00:00<00:00, 6172.41it/s]


In [6]:
df = pd.DataFrame(res)
df.link = df.link.apply(lambda x : URL + x if 'https' not in x else x)
df

,title,link
0,Dune: ¿Quiénes son los Harkonnen? La casa de l...,https://es.ign.com/dune-reboot/175693/feature/...
1,Primeras impresiones de Riders Republic: la di...,https://es.ign.com/riders-republic/175720/prev...
2,Vuelve Saints Row: sus creadores nos hablan de...,https://es.ign.com/saints-row/175539/feature/v...
3,"Así es Martha is Dead, el juego de terror que ...",https://es.ign.com/martha-is-dead/175583/previ...
4,Recrean el nuevo tráiler de Spider-Man: Sin Ca...,https://es.ign.com/spider-man-3-tom-holland/17...
...,...,...
75,No More Heroes 3: IGN Japón será el anfitrión ...,https://es.ign.com/no-more-heroes-1/175655/new...
76,Bethesda contrata a la guionista del mod Fallo...,https://es.ign.com/fallout-4/175654/news/bethe...
77,El sistema de combate clásico de Devil May Cry...,https://es.ign.com/devil-may-cry-hd-collection...
78,EA pone a disposición de todo el mundo cinco p...,https://es.ign.com/electronic-arts-1/175650/ne...


In [7]:
def get_article_data(url:str)-> str:
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    try:
        author = soup.find('span', class_='reviewer hcard').get_text()
    except : 
        author = None
    try:
        published_date = soup.find('div', class_='article-publish-date').get_text().split('el')[-1].strip()
    except: 
        published_date = soup.find('span', class_='publish-date').get_text()
    
    return author, published_date

In [24]:
print(df.link[3])
get_article_data(df.link[3])

https://es.ign.com/martha-is-dead/175583/preview/asi-es-martha-is-dead-el-juego-de-terror-que-nos-lleva-a-un-viaje-de-supersticion-y-traumas-psicolog


('Jose Alberto', '26 de agosto de 2021 a las 15:00')

In [25]:
df['additional_content'] = df.link.apply(lambda x : get_article_data(x))

  0%|          | 0/2 [03:53<?, ?it/s]


In [26]:
df

,title,link,additional_content
0,Dune: ¿Quiénes son los Harkonnen? La casa de l...,https://es.ign.com/dune-reboot/175693/feature/...,"(Ign, 26 de agosto de 2021 a las 21:23)"
1,Primeras impresiones de Riders Republic: la di...,https://es.ign.com/riders-republic/175720/prev...,"(Jose Alberto, 26 de agosto de 2021 a las 17:00)"
2,Vuelve Saints Row: sus creadores nos hablan de...,https://es.ign.com/saints-row/175539/feature/v...,"(David Oña, 25 de agosto de 2021 a las 22:01)"
3,"Así es Martha is Dead, el juego de terror que ...",https://es.ign.com/martha-is-dead/175583/previ...,"(Jose Alberto, 26 de agosto de 2021 a las 15:00)"
4,Recrean el nuevo tráiler de Spider-Man: Sin Ca...,https://es.ign.com/spider-man-3-tom-holland/17...,"(Jose Alberto, 27 de agosto de 2021 a las 12:13)"
...,...,...,...
75,No More Heroes 3: IGN Japón será el anfitrión ...,https://es.ign.com/no-more-heroes-1/175655/new...,"(Ign, 24 de agosto de 2021 a las 16:10)"
76,Bethesda contrata a la guionista del mod Fallo...,https://es.ign.com/fallout-4/175654/news/bethe...,"(Ign, 24 de agosto de 2021 a las 15:57)"
77,El sistema de combate clásico de Devil May Cry...,https://es.ign.com/devil-may-cry-hd-collection...,"(Ign, 24 de agosto de 2021 a las 15:39)"
78,EA pone a disposición de todo el mundo cinco p...,https://es.ign.com/electronic-arts-1/175650/ne...,"(Ign, 24 de agosto de 2021 a las 15:26)"


In [27]:
df['author'] = df.additional_content.apply(lambda x : x[0])
df['published_date'] = df.additional_content.apply(lambda x : x[1])
df = df.drop(['additional_content'], axis = 1)
df

,title,link,author,published_date
0,Dune: ¿Quiénes son los Harkonnen? La casa de l...,https://es.ign.com/dune-reboot/175693/feature/...,Ign,26 de agosto de 2021 a las 21:23
1,Primeras impresiones de Riders Republic: la di...,https://es.ign.com/riders-republic/175720/prev...,Jose Alberto,26 de agosto de 2021 a las 17:00
2,Vuelve Saints Row: sus creadores nos hablan de...,https://es.ign.com/saints-row/175539/feature/v...,David Oña,25 de agosto de 2021 a las 22:01
3,"Así es Martha is Dead, el juego de terror que ...",https://es.ign.com/martha-is-dead/175583/previ...,Jose Alberto,26 de agosto de 2021 a las 15:00
4,Recrean el nuevo tráiler de Spider-Man: Sin Ca...,https://es.ign.com/spider-man-3-tom-holland/17...,Jose Alberto,27 de agosto de 2021 a las 12:13
...,...,...,...,...
75,No More Heroes 3: IGN Japón será el anfitrión ...,https://es.ign.com/no-more-heroes-1/175655/new...,Ign,24 de agosto de 2021 a las 16:10
76,Bethesda contrata a la guionista del mod Fallo...,https://es.ign.com/fallout-4/175654/news/bethe...,Ign,24 de agosto de 2021 a las 15:57
77,El sistema de combate clásico de Devil May Cry...,https://es.ign.com/devil-may-cry-hd-collection...,Ign,24 de agosto de 2021 a las 15:39
78,EA pone a disposición de todo el mundo cinco p...,https://es.ign.com/electronic-arts-1/175650/ne...,Ign,24 de agosto de 2021 a las 15:26


In [50]:
final_df = pd.concat([pd.read_csv('test.csv'),df], axis = 0).drop_duplicates().reset_index(drop = True)
final_df

,title,link,author,published_date
0,Llevo más de un año jugando a Animal Crossing ...,https://es.ign.com/animal-crossing-new-horizon...,Álex Pareja,5 de agosto de 2021 a las 11:15
1,¿Quién es Peacemaker? El personaje de John Cen...,https://es.ign.com/peacemaker/175241/feature/q...,Ign,5 de agosto de 2021 a las 09:04
2,Las 10 mejores películas de fantasía de la his...,https://es.ign.com/movies/175174/feature/las-1...,David Oña,4 de agosto de 2021 a las 09:00
3,Hemos probado la Beta abierta de Back 4 Blood:...,https://es.ign.com/back-4-blood/175143/preview...,Ign,2 de agosto de 2021 a las 18:00
4,Los creadores de South Park firman un gran acu...,https://es.ign.com/south-park-1/175253/news/lo...,Ign,5 de agosto de 2021 a las 16:32
...,...,...,...,...
154,No More Heroes 3: IGN Japón será el anfitrión ...,https://es.ign.com/no-more-heroes-1/175655/new...,Ign,24 de agosto de 2021 a las 16:10
155,Bethesda contrata a la guionista del mod Fallo...,https://es.ign.com/fallout-4/175654/news/bethe...,Ign,24 de agosto de 2021 a las 15:57
156,El sistema de combate clásico de Devil May Cry...,https://es.ign.com/devil-may-cry-hd-collection...,Ign,24 de agosto de 2021 a las 15:39
157,EA pone a disposición de todo el mundo cinco p...,https://es.ign.com/electronic-arts-1/175650/ne...,Ign,24 de agosto de 2021 a las 15:26


In [51]:
final_df.to_csv('test.csv', index = False)